<a href="https://colab.research.google.com/github/erhansozen/Assignments-LearningPortfolio/blob/main/Assignment_7_Part_2_Erhan_S%C3%B6zen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following assignment consists again of a theoretical part (learning portfolio) and a practical part (assignment). The goal is to train a neural model for a recommendation system.

The plan would be that in the first week we will discuss your learnings from the theory part, that means you are relatively free to fill your Learning Portfolio on this topic and in the following week we will discuss your solutions of the practical part.

#Theory part (filling your Learning Portfolio, June 7)

In preparation for the practical part, I ask you to familiarize yourself with the following video sources in the next week:

1) Please watch the following videos:

https://www.youtube.com/watch?v=Fmtorg_dmM0&ab_channel=ritvikmath (not absolutely necessary, only for the overview)

https://course.fast.ai/Lessons/lesson7.html (The second part of the presentation starting with the topic collaborative filtering is mandatory)

Note: The first part of the video mainly contains tips for neural networks to submit a Kaggle Competition. For that, you would have to watch the end of the 6th video to understand this better. But this is not mandatory.

2) Please download the following notebooks and edit it in Google-Colab. Try to answer a few questions that are asked at the end. Take notes and update your Learning Portfolio.

https://www.kaggle.com/code/jhoward/collaborative-filtering-deep-dive/notebook


#Practical part (Assignment, June 14)

Find any data set that can be used for a recommender system and try to train and validate a neural network for it.

For this purpose I ask you to download a data set from the given lists and to use it for your program application.

https://gist.github.com/entaroadun/1653794

https://github.com/caserec/Datasets-for-Recommender-Systems

https://grouplens.org/datasets/movielens/

https://eigentaste.berkeley.edu/dataset/

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout
from google.colab import drive #allows us to reach our google drive
drive.mount('/content/drive')

# Veri setini yükle/ Dataset upload
movies = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ratings.csv')

# İlgili özellikleri seç / Related columns of dataset
movies = movies[['movieId', 'title']]
ratings = ratings[['userId', 'movieId', 'rating']]

# LabelEncoder kullanarak kullanıcı ve film kimliklerini dönüştür / user and movie ID's transforming by label encoder
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
ratings['userId'] = user_encoder.fit_transform(ratings['userId'])
ratings['movieId'] = movie_encoder.fit_transform(ratings['movieId'])

# Veri setini eğitim ve test olarak böle / Train and test data set splitting
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

# Giriş katmanı / Input layer
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

# Kullanıcıları ve filmleri temsil etmek için gömme katmanları / Embedding layers to represent users and movies
user_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=10)(user_input)
movie_embedding = Embedding(input_dim=len(movie_encoder.classes_), output_dim=10)(movie_input)

# Gömme vektörlerini düzleştir / flatting of user and movies vectors
user_flat = Flatten()(user_embedding)
movie_flat = Flatten()(movie_embedding)

# Özniteliklerin birleştiği gizli katmanlar / Hidden layers where the features merging
hidden = Dense(64, activation='relu')(user_flat)
hidden = Dropout(0.2)(hidden)
hidden = Dense(32, activation='relu')(hidden)
hidden = Dropout(0.2)(hidden)

# Tahmin katmanı / Estimation layer
output = Dense(1, activation='linear')(hidden)

# Modeli oluştur / Model creating
model = Model(inputs=[user_input, movie_input], outputs=output)
model.compile(loss='mean_squared_error', optimizer='adam')

# Modeli eğit / Model training (I chose the 2 epochs to quicker training time :'))
model.fit([train['userId'], train['movieId']], train['rating'], batch_size=64, epochs=2, verbose=1)

# Test veri seti üzerinde performansı değerlendir / Evaluation of performance on test dataset
mse = model.evaluate([test['userId'], test['movieId']], test['rating'])
print("Mean Squared Error:", mse)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/2
312502/312502 [==============================] - 6317s 20ms/step - loss: 0.9556
Epoch 2/2
156251/156251 [==============================] - 232s 1ms/step - loss: 0.9258
Mean Squared Error: 0.9257540702819824
